# Day 8: Part 2

#### Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). 
### What is the value of the accumulator after the program terminates?

In [40]:
with open('test8.txt') as f:
    text = f.readlines()
    text = [line.strip() for line in text]

print(text)

['nop +0', 'acc +1', 'jmp +4', 'acc +3', 'jmp -3', 'acc -99', 'acc +1', 'jmp -4', 'acc +6']


In [41]:
text = [item.split(' ') for item in text]

In [42]:
text

[['nop', '+0'],
 ['acc', '+1'],
 ['jmp', '+4'],
 ['acc', '+3'],
 ['jmp', '-3'],
 ['acc', '-99'],
 ['acc', '+1'],
 ['jmp', '-4'],
 ['acc', '+6']]

In [4]:
import pandas as pd
input_df = pd.DataFrame(text, columns=['type', 'amount'])
input_df

,type,amount
0,nop,+0
1,acc,+1
2,jmp,+4
3,acc,+3
4,jmp,-3
5,acc,-99
6,acc,+1
7,jmp,-4
8,acc,+6


In [5]:
input_df['visited?'] = 0
input_df

,type,amount,visited?
0,nop,+0,0
1,acc,+1,0
2,jmp,+4,0
3,acc,+3,0
4,jmp,-3,0
5,acc,-99,0
6,acc,+1,0
7,jmp,-4,0
8,acc,+6,0


**Write a function to interpret each line**

In [7]:
accumulator = 0
instruction = ''
sign = ''
amount = 0

In [8]:
def uncorrupt(i):
    instruction = input_df.iloc[i]['type']
    sign = input_df.iloc[i]['amount'][0]
    amount = int(input_df.iloc[i]['amount'][1:])
    
    if instruction == 'nop':
        input_df.at[i, 'type'] = 'jmp'
    if instruction == 'jmp':
        input_df.at[i, 'type'] = 'nop'
    handheld(i)

In [10]:
def handheld(i):
    global accumulator, instruction, sign, amount
    # First check if you've already visited this row
    # If you have, you're done! Return the accumulator
    if input_df.iloc[i]['visited?'] == 1:
        uncorrupt(i)
    
    # Get the instruction and amount
    instruction = input_df.iloc[i]['type']
    sign = input_df.iloc[i]['amount'][0]
    amount = int(input_df.iloc[i]['amount'][1:])
        
    # Follow the instructions!
    if instruction == 'nop':
        input_df.at[i, 'visited?'] = 1
        handheld(i+1)
        
    elif instruction == 'acc':
        input_df.at[i, 'visited?'] = 1
        if sign == '+':
            accumulator += amount
            handheld(i+1)
        if sign == '-':
            accumulator -= amount
            handheld(i+1)
        
    elif instruction == 'jmp':
        input_df.at[i, 'visited?'] = 1
        if sign == '+':
            handheld(i+amount)
        if sign == '-':
            handheld(i-amount)

In [11]:
handheld(0)

RecursionError: maximum recursion depth exceeded while calling a Python object

***Currently, my idea is to call the previous frame on the stack (since handheld() is a recursive function) and then access the previous value of i from that...but there's probably an easier way (using global variables? maybe make one called 'last i'?)***

Back about a year later! I actually think all I have to do is run the same program as for Part 1, but try it after changing different `nop` and `jmp` lines to see which one terminates.

In [12]:
accumulator = 0

def handheld(i):
    global accumulator
    # First check if you've already visited this row
    # If you have, you're done! Return the accumulator
    if input_df.iloc[i]['visited?'] == 1:
        return False
    
    # Get the instruction and amount
    instruction = input_df.iloc[i]['type']
    sign = input_df.iloc[i]['amount'][0]
    amount = int(input_df.iloc[i]['amount'][1:])
        
    # Follow the instructions!
    if instruction == 'nop':
        input_df.at[i, 'visited?'] = 1
        handheld(i+1)
        
    elif instruction == 'acc':
        input_df.at[i, 'visited?'] = 1
        if sign == '+':
            accumulator += amount
            handheld(i+1)
        if sign == '-':
            accumulator -= amount
            handheld(i+1)
        
    elif instruction == 'jmp':
        input_df.at[i, 'visited?'] = 1
        if sign == '+':
            handheld(i+amount)
        if sign == '-':
            handheld(i-amount)

In [33]:
text

[['jmp', '+0'],
 [-1, '+1'],
 [-1, '+4'],
 [-1, '+3'],
 [-1, '-3'],
 ['acc', '-99'],
 [-1, '+1'],
 [-1, '-4'],
 ['acc', '+6']]

In [14]:
int('+4')

4

In [15]:
int(-4)

-4

lol, didn't even need to bother with all that 'sign' business!

In [50]:
from copy import deepcopy

def program(boot_code):
    edit_code = deepcopy(boot_code)
    accumulator = 0
    i = 0
    
    while i<len(boot_code):
        instruction = edit_code[i]
        
        if instruction[0] == -1:
            print('Found infinite loop!')
            return False
        
        elif instruction[0] == 'nop':
            i += 1
            instruction[0] = -1
            continue
        
        elif instruction[0] == 'acc':
            accumulator += int(instruction[1])
            i += 1
            instruction[0] = -1
            continue

        elif instruction[0] == 'jmp':
            i += int(instruction[1])
            instruction[0] = -1
            continue
        else:
            continue
    
    return accumulator

def solution(text):
    boot_code = deepcopy(text)
    
    for code in boot_code:
        if code[0] == 'jmp':
            code[0] = 'nop'
            if program(boot_code):
                return program(boot_code)
            else:
                code[0] = 'jmp'
                
                
        elif code[0] == 'nop':
            code[0] = 'jmp'
            if program(boot_code):
                return program(boot_code)
            else:
                code[0] = 'nop'
                

In [51]:
with open('test8.txt') as f:
    text = f.readlines()
    text = [line.strip() for line in text]
text = [item.split(' ') for item in text]
text

[['nop', '+0'],
 ['acc', '+1'],
 ['jmp', '+4'],
 ['acc', '+3'],
 ['jmp', '-3'],
 ['acc', '-99'],
 ['acc', '+1'],
 ['jmp', '-4'],
 ['acc', '+6']]

In [52]:
solution(text)

Found infinite loop!
Found infinite loop!
Found infinite loop!


8

In [53]:
text

[['nop', '+0'],
 ['acc', '+1'],
 ['jmp', '+4'],
 ['acc', '+3'],
 ['jmp', '-3'],
 ['acc', '-99'],
 ['acc', '+1'],
 ['jmp', '-4'],
 ['acc', '+6']]

In [54]:
with open('input8.txt') as f:
    text = f.readlines()
    text = [line.strip() for line in text]
text = [item.split(' ') for item in text]
text

[['nop', '+456'],
 ['nop', '+38'],
 ['acc', '+9'],
 ['jmp', '+153'],
 ['acc', '+15'],
 ['nop', '+560'],
 ['jmp', '+452'],
 ['acc', '+26'],
 ['acc', '+42'],
 ['jmp', '+376'],
 ['acc', '-5'],
 ['acc', '+12'],
 ['acc', '-5'],
 ['jmp', '+15'],
 ['jmp', '+1'],
 ['acc', '-9'],
 ['jmp', '+533'],
 ['acc', '+19'],
 ['acc', '+33'],
 ['acc', '+34'],
 ['jmp', '-6'],
 ['nop', '+404'],
 ['nop', '+140'],
 ['acc', '+0'],
 ['jmp', '+123'],
 ['acc', '+45'],
 ['acc', '+0'],
 ['jmp', '+496'],
 ['jmp', '+487'],
 ['acc', '+9'],
 ['acc', '+34'],
 ['jmp', '+484'],
 ['acc', '+0'],
 ['acc', '-14'],
 ['jmp', '+466'],
 ['acc', '+40'],
 ['acc', '+6'],
 ['acc', '+30'],
 ['jmp', '+444'],
 ['nop', '+386'],
 ['jmp', '+215'],
 ['acc', '+43'],
 ['acc', '+5'],
 ['nop', '-4'],
 ['jmp', '+535'],
 ['jmp', '-13'],
 ['acc', '+3'],
 ['acc', '+7'],
 ['acc', '+49'],
 ['acc', '-1'],
 ['jmp', '+245'],
 ['acc', '+9'],
 ['acc', '+31'],
 ['nop', '+142'],
 ['jmp', '+554'],
 ['acc', '+3'],
 ['jmp', '+493'],
 ['nop', '+399'],
 ['jmp', '

In [55]:
solution(text)

Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinite loop!
Found infinit

780